# Introduction

This workshop will go over some ways to think about optimizing kernels. We'll do
this through the lens of matrix multiplication. 

Matrix multiplication is a good candidate to learn the fundamentals of kernel
optimization for a number of reasons:

1. Matrix multiplication is "compute bound", meaning the speed at which a chip
   can perform floating point operations is the limiting factor
2. Matrix multiplication is straightforward to implement and think about
3. Matrix multiplication is widely used, and other linear algebra operations can
   be recast as matrix multiplication
   (see: [Matricization](https://en.wikipedia.org/wiki/Tensor_reshaping#Mode-m_Flattening_/_Mode-m_Matrixization))


# Quick GPU architecture overview

NVIDIA Hopper chip (GH100):

<img src="../resources/hopper-full-gpu.png" width=800 />

- Image taken from [NVIDIA Hopper architecture blogpost](https://developer.nvidia.com/blog/nvidia-hopper-architecture-in-depth/)


## Simplified view:
<img src="../resources/CPUGPU.png" width=500 />

- Image taken from the [CUDA programming guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html)

- GPUs trade fancy features for massive amounts of compute units
- CPUs trade compute for fancy features (think branch prediction)

## A closer look at GPUs: Streaming Multiprocessors (SMs)

NVIDIA Hopper SM architecture (GH100):

<img src="../resources/hopper-sm-arch.png" width=500 />

- Image taken from [NVIDIA Hopper architecture blogpost](https://developer.nvidia.com/blog/nvidia-hopper-architecture-in-depth/)

- SMs are the "core" of the GPU
    - Commonly confused with the compute units ("CUDA cores")
- Each GPU has a number of SMs
    - H100 SM count: 114
    - A100 SM count: 108
- Getting "performance" out of a GPU usually means writing code in such a way
  that SMs are as busy as possible and running as efficiently as possible with
  respect to memory accesses

# Matrix multiplication

Given $A \in \mathbb{R}^{n \times n}, B \in \mathbb{R}^{n \times n}$ and
$C \in \mathbb{R}^{n \times n}$
$$
C_{ij} = \sum_{k=1}^n A_{ik} B_{kj}
$$

# Runner utility
To make it so we don't need to write a driver in C++, we have some helper classes
that will use [`cupy`]() to run the kernels from our notebooks. There are blank
kernels where we will be filling in details like linearized index expressions and
loading into shared memory.

In [ ]:
import cupy as cu
import numpy as np

import os

from dataclasses import dataclass

@dataclass(frozen=True)
class KernelRunner:
    kernel_name: str
    template: str

    def _get_kernel_src(self, read_full_src: bool = False) -> str:
        cur_dir = os.getcwd()

        kernel_dir = "full" if read_full_src else "blank"

        with open(
            f"{cur_dir}/../src/kernels/{kernel_dir}/{self.kernel_name}.cu",
            "r") as f:
            src = f.read()

        return src

    def __call__(self,
                 block_dim: tuple[int, ...],
                 grid_dim: tuple[int, ...],
                 args: tuple,
                 read_full_src: bool = False,
                 niterations: int = 1) -> np.ndarray:

        N, = args

        src = self._get_kernel_src(read_full_src)

        wrapper_name = f"{self.kernel_name}_float"
        code = src + f"""
extern "C" __global__
void {wrapper_name}(const float *__restrict__ A,
                    const float *__restrict__ B,
                    float *__restrict__ C,
                    const int N) {{
  {self.kernel_name}_matmul{self.template}(A, B, C, N);
}}
        """

        mod = cu.RawModule(code=code,
                           options=("-std=c++14",),
                           name_expressions=[wrapper_name])

        func = mod.get_function(wrapper_name)

        A = cu.random.rand(N, N, dtype=cu.float32)
        B = cu.random.rand(N, N, dtype=cu.float32)
        C = cu.zeros_like(A)

        C_true = A @ B

        if niterations == 1:
            func(grid_dim, block_dim, (A, B, C, N))
            cu.cuda.runtime.deviceSynchronize()
            print(f"Error = {cu.linalg.norm(C_true - C) / cu.linalg.norm(C):.4f}")
        else:
            # warm up calls
            for _ in range(5):
                func(grid_dim, block_dim, (A, B, C, N))
            cu.cuda.runtime.deviceSynchronize()

            start = cu.cuda.Event()
            end = cu.cuda.Event()

            start.record()
            for _ in range(niterations):
                func(grid_dim, block_dim, (A, B, C, N))
            end.record()
            end.synchronize()

            ms = cu.cuda.get_elapsed_time(start, end)

            print(10*"=", self.kernel_name, 10*"=")
            print(f"Total time  : {ms:.4f} ms")
            print(f"Average time: {ms / niterations:.4f} ms")

            # FIXME: compute observed GFLOP/s
            
            # HINT: you'll need total flops, and to convert ms to s
            # per iteration
            
            # NOTE: this is just a rough estimate. if you want a better pulse on
            # how a kernel is performing, consider using NVIDIA Nsight Compute to
            # profile
            gflops = 0.0
            print(f"GFLOP/s     : {gflops:.4f}")
            print(f"Error       : {cu.linalg.norm(C_true - C) / cu.linalg.norm(C):.4f}")
            print((22 + len(self.kernel_name))*"=")

        return C.get()

In [ ]:
# define a custom runner:
@dataclass(frozen=True)
class NaiveKernelRunner(KernelRunner):
    template: str = "<float>"
    kernel_name: str = "naive"

# usage would be something like this:
N = 1024
BM = BN = 32
block_dim = (BM, BN)
grid_dim = (N // BM, N // BN)

runner = NaiveKernelRunner()

# NOTE: for kernels we're running, set read_full_src=False to use the kernel 
# you edit/write 
C = runner(block_dim, grid_dim, (N,), read_full_src=True, niterations=10)

# Data storage formats

Arrays are multidimensional, but memory is linear. There is no "right answer" to
how to linearize arrays in memory.

The two dominant ways of storing to memory are "row-major" (C-order, C for C) and 
"column-major" (F-order, F for Fortran).

For row major, we take the rows of our array and lay them out from beginning to
end, one after another.

For column major, we instead move down the columns one after another.

<img src="../../resources/array-layout.png" width=300 />

Our arrays are *row-major*.

In [7]:
import time
import numpy as np

N = 2048 
a = np.random.randn(N, N)
acc = 0.0
start = time.time()
# write a loop that accumulates a[i,j] using (i, j) loop order
end = time.time()
print(f"(i, j) order took: {end - start:.4f} s")

acc = 0.0
start = time.time()
# write a loop that accumulates a[i,j] using (j, i) loop order
end = time.time()
print(f"(j, i) order took: {end - start:.4f} s")

(i, j) order took: 0.7688 s
(j, i) order took: 1.0204 s


# Naive implementation and analysis

Go to <a href="../src/kernels/blank/naive.cu">the blank `naive.cu` kernel</a>. 
We'll start by writing the necessary pieces to get up and running. We'll need
to fill in all the `FIXME`s.

In [ ]:
# define the naive runner:
@dataclass(frozen=True)
class NaiveKernelRunner(KernelRunner):
    template: str = "<float>"
    kernel_name: str = "naive"

# the kernels do not have bounds checking to reduce visual noise, so we need to
# choose N, BM, and BN such that N is evenly divisible by BM and BN
# some values to try: 1024, 2048, 4096, 8192
N = ...
BN = BM = ...
block_dim = ...
grid_dim = ...
runner = ...
C = runner(block_dim, grid_dim, (N,), read_full_src=False, niterations=20)

In [ ]:
clock_speed_mhz = 1695.0
clocks_per_sec = ...
flops_per_clock = ...
flops_per_sec = ...

gb_per_sec = 936.2
bytes_per_sec = ...

arithmetic_intensity = flops_per_sec / bytes_per_sec